# READ_ME
学習済みモデルを使用し，予測をします．  
"models"ディレクトリをダウンロードしておく必要があります．   
以下のセルでmodelsを置いたディレクトリをdata_dirに指定し，   
`predict`を実行します．

In [1]:
###modelが置かれているディレクトリを，"/"まで含めて指定する
data_dir = "/root/userspace/CIFAR10_final/models/"

In [ ]:
!pip3 install keras
import keras
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score

In [46]:
(x_train_raw,t_train_raw), (x_test_raw,t_test_raw) = cifar10.load_data()

In [47]:
x_train = x_train_raw /255
x_test = x_test_raw /255
t_train = to_categorical(t_train_raw)
t_test = to_categorical(t_test_raw)

In [45]:
batch_size = 500

def tta(model,test_size,generator,epochs = 10):
    #test_time_augmentation
    #batch_sizeは，test_sizeの約数でないといけない．
    pred = np.zeros(shape = (test_size,10), dtype = float)
    step_per_epoch = test_size //batch_size
    for epoch in range(epochs):
        for step in range(step_per_epoch):
            sta = batch_size * step
            end = sta + batch_size
            tmp_x = generator.__next__()
            pred[sta:end] += model.predict(tmp_x)
    return pred / epochs


def tta_generator():
    return ImageDataGenerator(rotation_range = 20 , horizontal_flip = True,height_shift_range = 0.2,\
                                 width_shift_range = 0.2,zoom_range = 0.2,channel_shift_range = 0.2\
                                  ).flow(x_test,batch_size = batch_size,shuffle = False)

# 学習済みモデルを使用し，予測をする関数
def predict(x,t= [],tta_epochs = 30):
    #xは，予測したい画像データ
    #tは，教師データ．指定された場合accuracyを標準出力する
    #batch_sizeは，TTAで画像を生成する際のバッチサイズ．予測データの枚数の約数にしないとバグるかも
    #tta_epochsは，一つの画像に対して何枚の画像をAugmentationにより生成するか
    #返り値：　予測データ
    pred = np.zeros( (len(x),10))
    for i in range(10):
        model = load_model(data_dir + "/ensemble_" + str(i) + ".hdf5" )
        tmp_pred = tta(model,x_test.shape[0],tta_generator() ,epochs = tta_epochs)
        pred += tmp_pred
        print(i+1,end ="")
        print("回目終了")
        if( len(t) >0):
            print("accuracy: ",end = "")
            print(accuracy_score( np.argmax( t,axis = 1,),np.argmax(tmp_pred,axis =1)) )
    if( len(t) > 0):
        print("final acc: ",end = "")
        print(accuracy_score( np.argmax( t,axis = 1,),np.argmax(pred,axis =1)) )
    return pred

In [ ]:
predict(x_test,t_test,tta_epochs = 10)